# [TF.Text](https://www.tensorflow.org/tutorials/tensorflow_text/intro)

TensorFlow Text はテキストデータに関するクラスとメソッドを提供している。ライブラリはテキストの事前処理などのTensorFlowの大元が提供していないこともできる。TensorFlow graph を使えることも利点である。TensorFlow Text を使っていれば、トークン化や事前処理などを心配する必要がない。

In [1]:
import tensorflow as tf
import tensorflow_text as text

In [2]:
# Most ops expect that the strings are in UTF-8.
# If you're using a different encoding, you can use the core tensorflow transcode op to transcode into UTF-8.
docs = tf.constant([u'Everything not saved will be lost.'.encode('UTF-16-BE'), u'Sad☹'.encode('UTF-16-BE')])

# You can also use the same op to coerce your string to structurally valid UTF-8 if your input could be invalid.
utf8_docs = tf.strings.unicode_transcode(docs, input_encoding='UTF-16-BE', output_encoding='UTF-8')

## トークン化

文字列をトークン（単語、数字、句読点）に分割する。`Tokenizer`や`TokenizerWithOffsets`インターフェースを使う。これらはそれぞれメソッドとして、`tokenize`と`tokenize_with_offsets`を持つ。いくつかのトークン化手法を利用することができ、それぞれ`TokenizerWithOffsets`（`Tokenizer`の拡張）を実装している。これらはそれぞれバイトオフセットを元の文字列に取り入れるオプションを含み、これを使うことによって、トークンが作成された元の文字列にあるバイトを知ることができる。

すべてのトークナイザーは、元の個々の文字列にマッピングされたトークンの最も内側の次元を持つ`RaggedTensor`を返します。 その結果、結果のシェイプのランクが1つ上がります。 それらに慣れていない場合は、[ragged_tensors](https://www.tensorflow.org/guide/ragged_tensors)を確認してください。

In [3]:
# Split UTF-8 strings on ICU defined whitespace characters (e.g. space, tab, newline).
tokenizer = text.WhitespaceTokenizer()
tokens = tokenizer.tokenize(['everything not saved will be lost.', u'Sad☹'.encode('UTF-8')])
print(tokens.to_list())

Instructions for updating:
`tf.batch_gather` is deprecated, please use `tf.gather` with `batch_dims=-1` instead.
[[b'everything', b'not', b'saved', b'will', b'be', b'lost.'], [b'Sad\xe2\x98\xb9']]


In [4]:
# Split UTF-8 strings based on Unicode script boundaries.
tokenizer = text.UnicodeScriptTokenizer()
tokens = tokenizer.tokenize(['everything not saved will be lost.', u'Sad☹'.encode('UTF-8')])
print(tokens.to_list())

[[b'everything', b'not', b'saved', b'will', b'be', b'lost', b'.'], [b'Sad', b'\xe2\x98\xb9']]


In [5]:
# Split by character (common to use when tokenizing languages without whitespace).
tokens = tf.strings.unicode_split([u"仅今年前".encode('UTF-8')], 'UTF-8')
print(tokens.to_list())

[[b'\xe4\xbb\x85', b'\xe4\xbb\x8a', b'\xe5\xb9\xb4', b'\xe5\x89\x8d']]


In [6]:
tokenizer = text.UnicodeScriptTokenizer()

# Since it is desired to know where in the original string the token originated from,
# each tokenizer which implements `TokenizerWithOffsets` has a `tokenize_with_offsets`method
# that will return the byte offsets along with the tokens.
(tokens, start_offsets, end_offsets) = tokenizer.tokenize_with_offsets([
    'everything not saved will be lost.',
    u'Sad☹'.encode('UTF-8')
])

print(tokens.to_list())
# `start_offsets` lists the bytes in the original string each token starts at (inclusive).
print(start_offsets.to_list())
# `end_offsets` lists the bytes immediately after the point where each token ends (exclusive).
print(end_offsets.to_list())

[[b'everything', b'not', b'saved', b'will', b'be', b'lost', b'.'], [b'Sad', b'\xe2\x98\xb9']]
[[0, 11, 15, 21, 26, 29, 33], [0, 3]]
[[10, 14, 20, 25, 28, 33, 34], [3, 6]]


In [7]:
# example
docs = tf.data.Dataset.from_tensor_slices([['Never tell me the odds.'], ["It's a trap!"]])
tokenizer = text.WhitespaceTokenizer()
tokenized_docs = docs.map(lambda x: tokenizer.tokenize(x))
iterator = iter(tokenized_docs)
print(next(iterator).to_list())
print(next(iterator).to_list())

[[b'Never', b'tell', b'me', b'the', b'odds.']]
[[b"It's", b'a', b'trap!']]


## そのほか

TF.Text パッケージはほかにも便利な事前処理方法を提供している。

文章の分割モデルは文字が大文字なのか小文字なのか、句読点が文末なのか文中なのかなどの特徴を見る場合がある。`Wordshape`は入力に対して多くのパターンをマッチングさせるための機能を持つ。

In [8]:
tokenizer = text.WhitespaceTokenizer()
tokens = tokenizer.tokenize(['Everything not saved will be lost.', u'Sad☹'.encode('UTF-8')])

# Is capitalized?
f1 = text.wordshape(tokens, text.WordShape.HAS_TITLE_CASE)
# Are all letters uppercased?
f2 = text.wordshape(tokens, text.WordShape.IS_UPPERCASE)
# Does the token contain punctuation?
f3 = text.wordshape(tokens, text.WordShape.HAS_SOME_PUNCT_OR_SYMBOL)
# Is the token a number?
f4 = text.wordshape(tokens, text.WordShape.IS_NUMERIC_VALUE)

print(f1.to_list())
print(f2.to_list())
print(f3.to_list())
print(f4.to_list())

[[True, False, False, False, False, False], [True]]
[[False, False, False, False, False, False], [False]]
[[False, False, False, False, False, True], [True]]
[[False, False, False, False, False, False], [False]]


N-gramsも利用することができる。これはサイズ$n$のスライディングウィンドウが与えられた時に得られる単語の配列である。トークンを組み合わせる時、三つの削減メカニズムがサポートされている。テキストについては、`Reduction.STRING_JOIN`を使って文字列を結合することができる。デフォルトのセパレータはスペースである。他の削減メカニズム`Reduction.SUM`と`Reduction.MEAN`は主に数値の場合に利用する。

In [9]:
tokenizer = text.WhitespaceTokenizer()
tokens = tokenizer.tokenize(['Everything not saved will be lost.', u'Sad☹'.encode('UTF-8')])

# Ngrams, in this case bi-gram (n = 2)
bigrams = text.ngrams(tokens, 2, reduction_type=text.Reduction.STRING_JOIN)

print(bigrams.to_list())

[[b'Everything not', b'not saved', b'saved will', b'will be', b'be lost.'], []]
